# Step 3: Model Operationalization & Deployment

In the previous script, you learned how to save lstm trained models to files. You also learned that model weights are easily stored using  HDF5 format and that the network structure can be saved in JSON. In this script, you will learn how to load your models up, operationalize and use them to make future predictions.

In [1]:
import json
import keras
import numpy as np
# import the libraries
import os
import pandas as pd
import shutil
import urllib
from keras.models import model_from_json

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
#from keras.layers import Dense, Dropout, LSTM, Activation gdt

import h5py
from sklearn import datasets

# Setup the pyspark environment
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, PipelineModel
spark = SparkSession.builder.getOrCreate()


# Use the Azure Machine Learning data collector to log various metrics
from azureml.logging import get_azureml_logger

# For Azure blob storage access
from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess

Using TensorFlow backend.


In [2]:
from azureml.logging import get_azureml_logger
run_logger = get_azureml_logger()
run_logger.log('amlrealworld.predictivemaintenanceforpm.operationalization','true')

In [3]:
# Enter your Azure blob storage details here 
ACCOUNT_NAME = "<your blob storage account name>"

# You can find the account key under the _Access Keys_ link in the 
# [Azure Portal](portal.azure.com) page for your Azure storage container.
ACCOUNT_KEY = "<your blob storage account key>"

#-------------------------------------------------------------------------------------------
# The data from the Data Ingestion and Preparation notebook is stored in the sensordata ingestion container.
CONTAINER_NAME = "sensordataingestionpm"

# Connect to your blob service     
az_blob_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

# We will store and read each of these data sets in blob storage in an 
# Azure Storage Container on your Azure subscription.
# See https://github.com/Azure/ViennaDocs/blob/master/Documentation/UsingBlobForStorage.md
# for details.

# This is the final feature data file.
TRAIN_DATA = 'PM_train_files.parquet'
TEST_DATA = 'PM_test_files.parquet'

# This is where we store the final model data file.
LOCAL_DIRECT = 'model_result.parquet'

In [4]:
# load the previous created final dataset into the workspace
# create a local path where we store results
if not os.path.exists(TEST_DATA):
    os.makedirs(TEST_DATA)
    print('DONE creating a local directory!')

# download the entire parquet result folder to local path for a new run 
for blob in az_blob_service.list_blobs(CONTAINER_NAME):
    if TRAIN_DATA in blob.name:
        local_file = os.path.join(TEST_DATA, os.path.basename(blob.name))
        az_blob_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)
        
test_df = spark.read.parquet(TEST_DATA)
test_df = test_df.toPandas()
test_df.head(10)

DONE creating a local directory!


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,RUL,label1,label2,cycle_norm
0,52,119,0.643678,0.083333,0.0,0.0,0.605422,0.424024,0.482613,0.0,...,0.0,0.500000,0.0,0.0,0.472868,0.453604,94,0,0,0.326870
1,52,120,0.189655,0.833333,0.0,0.0,0.409639,0.440811,0.392302,0.0,...,0.0,0.416667,0.0,0.0,0.527132,0.530793,93,0,0,0.329640
2,52,121,0.500000,0.750000,0.0,0.0,0.572289,0.507739,0.597907,0.0,...,0.0,0.500000,0.0,0.0,0.449612,0.544187,92,0,0,0.332410
3,52,122,0.517241,0.333333,0.0,0.0,0.466867,0.432963,0.492404,0.0,...,0.0,0.416667,0.0,0.0,0.387597,0.603010,91,0,0,0.335180
4,52,123,0.689655,0.750000,0.0,0.0,0.400602,0.468062,0.501519,0.0,...,0.0,0.250000,0.0,0.0,0.496124,0.540182,90,0,0,0.337950
5,52,124,0.747126,0.666667,0.0,0.0,0.484940,0.384783,0.493079,0.0,...,0.0,0.416667,0.0,0.0,0.589147,0.502900,89,0,0,0.340720
6,52,125,0.505747,0.750000,0.0,0.0,0.406627,0.349248,0.415429,0.0,...,0.0,0.500000,0.0,0.0,0.565891,0.628556,88,0,0,0.343490
7,52,126,0.448276,0.333333,0.0,0.0,0.463855,0.325921,0.455773,0.0,...,0.0,0.583333,0.0,0.0,0.410853,0.552886,87,0,0,0.346260
8,52,127,0.362069,0.083333,0.0,0.0,0.620482,0.373229,0.513504,0.0,...,0.0,0.416667,0.0,0.0,0.457364,0.551505,86,0,0,0.349030
9,52,128,0.500000,0.166667,0.0,0.0,0.379518,0.369087,0.392640,0.0,...,0.0,0.500000,0.0,0.0,0.527132,0.524855,85,0,0,0.351801


In [5]:
with open(os.environ['AZUREML_NATIVE_SHARE_DIRECTORY'] + 'modellstm.json', 'r') as json_file:
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    print("json file read from shared folder")

json file read from shared folder


In [6]:
with open(os.environ['AZUREML_NATIVE_SHARE_DIRECTORY'] + 'modellstm.h5', 'r') as model:
    loaded_model.load_weights(os.path.join(os.environ['AZUREML_NATIVE_SHARE_DIRECTORY'], 'modellstm.h5'))
    print("model weights read from shared folder")

model weights read from shared folder


### Test init() and run() functions to read from the working directory

In [7]:
# pick a large window size of 50 cycles
sequence_length = 50

In [8]:
def init():
    # read in the model file
    from keras.models import model_from_json

    # load json and create model
    with open(os.environ['AZUREML_NATIVE_SHARE_DIRECTORY'] + 'modellstm.json', 'r') as json_file:
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    with open(os.environ['AZUREML_NATIVE_SHARE_DIRECTORY'] + 'modellstm.h5', 'r') as model:
        print("Loaded model")
    
    #inputs_dc = ModelDataCollector("modellstm.h5", identifier="inputs")
    #print(inputs_dc)
    #prediction_dc = ModelDataCollector("modellstm.h5", identifier="prediction")


In [9]:
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

In [10]:
# pick the feature columns 
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
input_features = sensor_cols + sequence_cols
input_features
sequence_cols.extend(sensor_cols)

In [11]:
# generator for the sequences
seq_gen = (list(gen_sequence(test_df[test_df['id']==id], sequence_length, sequence_cols)) 
           for id in test_df['id'].unique())

In [12]:
# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

(15631, 50, 25)

In [13]:
def run(seq_array): 
    global clf2, inputs_dc, prediction_dc
    try:
        prediction = loaded_model.predict_proba(seq_array)
        #print(prediction)
        #inputs_dc.collect(seq_array)
        #prediction_dc.collect(prediction)
        return (prediction)
    except Exception as e:
        return(str(e))

In [14]:
prediction = loaded_model.predict_proba(seq_array)
prediction

array([[0.02161   ],
       [0.0240796 ],
       [0.02850101],
       ...,
       [0.9828591 ],
       [0.98365027],
       [0.9840912 ]], dtype=float32)

In [15]:
init()

Loaded model


In [16]:
run(seq_array)

array([[0.02161   ],
       [0.0240796 ],
       [0.02850101],
       ...,
       [0.9828591 ],
       [0.98365027],
       [0.9840912 ]], dtype=float32)

## Persist model assets

Next we persist the assets we have created to disk for use in operationalization.

In [17]:
%%writefile {os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']}/lstmscore.py

import keras
# import the libraries
import os
import pandas as pd
import numpy as np
import urllib
import json
import shutil

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation

def init():
    # read in the model file
    from keras.models import model_from_json

    # load json and create model
    json_file = open('modellstm.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("modellstm.h5")
    print("Loaded model")
    
    #inputs_dc = ModelDataCollector("modellstm.h5", identifier="inputs")
    #print(inputs_dc)
    #prediction_dc = ModelDataCollector("modellstm.h5", identifier="prediction")
    
    
def run(seq_array): 
    global clf2, inputs_dc, prediction_dc
    try:
        prediction = loaded_model.predict_proba(seq_array)
        #print(prediction)
        #inputs_dc.collect(seq_array)
        #prediction_dc.collect(prediction)
        return (prediction)
    except Exception as e:
        return(str(e))

Overwriting /azureml-share//lstmscore.py


Retrieving the files from the shared folder:
https://docs.microsoft.com/en-us/azure/machine-learning/preview/how-to-read-write-files

## Deployment

Once the assets are stored, we can download them into a local compute context for operationalization on an Azure web service.

We demonstrate how to setup this web service this through a CLI window opened in the AML Workbench application. 

Once downloaded, unzip the file into the directory of your choosing. The zip file contains three deployment assets:

- the `lstmscore.py` file
- a `lstm.model` directory
- the `modellstm.json` file



## Create a model management endpoint 

Create a modelmanagement under your account. We will call this `lstmmodelmanagement`. The remaining defaults are acceptable.

`az ml account modelmanagement create --location <ACCOUNT_REGION> --resource-group <RESOURCE_GROUP> --name pdmmodelmanagement`


## Check environment settings

Show what environment is currently active:

`az ml env show`

If nothing is set, we setup the environment with the existing model management context first: 

` az ml env setup --location <ACCOUNT_REGION> --resource-group <RESOURCE_GROUP> --name pdmmodelmanagement`

then set the current environment:

`az ml env set --resource-group <RESOURCE_GROUP> --cluster-name pdmmodelmanagement`

Check that the environment is now set:

`az ml env show`


## Deploy your web service 

Once the environment is setup, we'll deploy the web service from the CLI.

These commands assume the current directory contains the webservice assets we created in throughout the notebooks in this scenario (`lstmscore.py`, `modellstm.json` and `lstm.model`). If your kernel has run locally, the assets will be in the `os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']`. 

On windows this points to:

```
cd C:\Users\<username>\.azureml\share\<team account>\<Project Name>
```

on linux variants this points to:

```
cd ~\.azureml\share\<team account>\<Project Name>
```


The command to create a web service (`<SERVICE_ID>`) with these operationalization assets in the current directory is:

```
az ml service create realtime -f <filename> -r <TARGET_RUNTIME> -m <MODEL_FILE> -s <SCHEMA_FILE> -n <SERVICE_ID> --cpu 0.1
```

The default cluster has only 2 nodes with 2 cores each. Some cores are taken for system components. AMLWorkbench asks for 1 core per service. To deploy multiple services into this cluster, we specify the cpu requirement in the service create command as (--cpu 0.1) to request 10% of a core. 

For this example, we will call our webservice `amlworkbenchpdmwebservice`. This `SERVICE_ID` must be all lowercase, with no spaces:

```
az ml service create realtime -f lstmscore.py -r spark-py -m lstm.model -s modellstm.json --cpu 0.1 -n amlworkbenchpdmwebservice
```

This command will take some time to execute. 

Once complete, the command returns sample usage commands to test the service for both PowerShell and the cmd prompt. We can execute these commands from the command line as well.